# EXPORTS JOBS POSTED FROM [web.byui.edu/StudentEmployment/’](web.byui.edu/StudentEmployment/) INTO A CSV FILE

In [ ]:
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup

## REQUEST DATA FROM API ENDPOINT

In [ ]:
conn = http.client.HTTPSConnection("web.byui.edu")
 
conn.request("GET", "/studentemployment/api/jobs")

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

In [ ]:
data_jobs['URL'] = data_jobs.jobID.apply(lambda x: f'https://web.byui.edu/StudentEmployment/job/{x}')

In [ ]:
data_jobs['URL'].iloc[300]

In [ ]:
data_jobs

In [ ]:
type(data)

## DISPLAY DATA RECEIVED

In [ ]:
pd.options.display.max_columns = None

data_jobs

## SIMPLE OVERVIEW

In [ ]:
print(f'''
    THERE ARE {data_jobs.jobID.nunique()} JOBS.
    HIGHEST PAY JOB:\'{data_jobs[data_jobs.payRate==data_jobs.payRate.max()]['title'].iloc[0]}\' WITH {data_jobs.payRate.max()} DOLLARS AN HOUR.
    ONLINE JOBS: {data_jobs[data_jobs["title"].str.contains('Online')].shape[0]} OUT OF {data_jobs.shape[0]}.
''')

## CLEAN THE DATA

In [ ]:
col_dates = ['dateUpdated','startDate','endDate','beginningDate','recruitingStartDate']

data_jobs[col_dates] = data_jobs[col_dates].astype('datetime64[ns]')

columns_to_drop = [
    'jobID', # not needed for EDA
    # 'description', # not needed for EDA
    # 'summary', # not needed for EDA
    'displayJob', # single boolean
    'dateUpdated','startDate','endDate', # not needed for display
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    # 'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'limitApplicants', # not relevant
    'limitNumber', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]

data_jobs.drop(columns_to_drop, axis=1, inplace=True)

data_jobs['description'] = data_jobs['description'].apply(lambda x: [p.text.strip() for p in BeautifulSoup(x).find_all('p') if p.text.strip() != ''])

## OPTIONAL: Save as a CSV

In [ ]:
# data_jobs.to_csv('StudentEmployment.csv')

## FILTER OUT THE JOBS THAT HAVE 'Online','Custodian', and 'TA' IN THEIR TITLE

In [ ]:
# JOBS FILTERED. NOT ONLINE, NOR CUSTODIAN, AND NOR TA.

remove = ['Online','Custodian','TA']

data_filtered = data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False).reset_index().drop('index',axis=1)

print(f'{data_filtered.shape[0]} JOBS OUT OF {data_jobs.shape[0]} POSTED JOBS.')

pd.set_option('display.max_rows', None)

data_filtered

## OPTIONAL: Save as a CSV

In [ ]:
# data_filtered.to_csv('StudentEmployment_filtered.csv')

In [ ]:
data_jobs.payRate.value_counts().reset_index().sort_values('index')

In [ ]:
data_jobs.groupby('payRate').count()

In [ ]:
data_jobs.departmentName.value_counts()

In [ ]:
data_jobs[data_jobs.title == 'AGBUS147 OnlineTA'].iloc[0]

In [7]:
pd.options.display.max_columns = None
data_jobs[data_jobs.jobID == 342319]

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,startDate,endDate,managerName,approximateHoursPerWeek,payRate,positionsAllocated,positionsAvailble,workSchedule,beginningDate,recruitingStartDate,requireResume,limitApplicants,limitNumber,applicants,jobQuestions,isOnline,allowOnline,jobMajors,URL
7,342319,PUBH370 OnlineTutor,Online Student,<p><span>Online Tutor is needed for the PUBH ...,An Online Tutor is needed for the PUBH 370 (H...,True,2022-09-06T21:26:02.6637842,2022-09-06T11:00:00,2022-09-13T08:00:00,Casey McDaniel,5-10 hours (must post 7 hours of appt openings...,10.0,100.0,99.0,Hours worked may vary according to student demand,2022-09-14T07:00:00,2019-05-09T01:00:00,True,True,50,[],[],False,True,[],https://web.byui.edu/StudentEmployment/job/342319
